# Tensor types in PaddlePaddle

本章我们主要对 PaddlePaddle 中的张量进行讲解。首先介绍 Fluid 中的 Tensor，然后介绍 LoD-Tensor 的相关知识。

PaddlePaddle 中使用的 Tensor 为 LoD-Tensor（Level-of-Detail Tensor）。它是 Fluid 中特有的概念，在普通 Tensor 的基础上附加了序列信息。Fluid中可传输的数据包括：输入、输出、网络中的可学习参数，全部统一使用 LoD-Tensor 表示。

在神经网络中传递的数据都是 Tensor, Tensor 可以简单理解成一个多维数组，一般而言可以有任意多的维度。不同的Tensor可以具有自己的数据类型和形状，同一 Tensor 中每个元素的数据类型是一样的，Tensor 的形状就是 Tensor 的维度。

在 Fluid 中存在三种特殊的 Tensor：

* **模型中的可学习参数**
* **输入输出Tensor**
* **常量 Tensor**

## Fluid 中的 Tensor

### 模型中的可学习参数

模型中的可学习参数（包括网络权重、偏置等）生存期和整个训练任务一样长，会接受优化算法的更新，在 Fluid 中以 Variable 的子类 Parameter 表示。

在Fluid中可以通过 `fluid.layers.create_parameter` 来创建可学习参数，例如通过下面程序来创建一个权重参数：

In [1]:
import paddle
import paddle.fluid as fluid

In [2]:
w = fluid.layers.create_parameter(name='w', shape=[1], dtype='float32')
print(w)

name: "w"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP32
      dims: 1
    }
  }
}
persistable: true



一般情况下，不需要自己来创建网络中的可学习参数，Fluid 为大部分常见的神经网络基本计算模块都提供了封装。以最简单的全连接模型为例，下面的代码片段会直接为全连接层创建连接权值（W）和偏置（ bias ）两个可学习参数，无需显式地调用 Parameter 相关接口来创建。

In [3]:
var_program = fluid.Program()
# define a Program to save fc operator
with fluid.program_guard(var_program, fluid.Program()):
    x = fluid.layers.data(name='x', shape=[13], dtype='float32')
    print('variables before fc operation.')
    params = [v for v in var_program.list_vars() if isinstance(v, fluid.framework.Parameter)]
    print(params)
    y = fluid.layers.fc(input=x, size=128, bias_attr=True)
    print('-'*30)
    print('variables after fc operation')
    params = [v for v in var_program.list_vars() if isinstance(v, fluid.framework.Parameter)]
    print(params)


variables before fc operation.
[]
------------------------------
variables after fc operation
[name: "fc_0.w_0"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP32
      dims: 13
      dims: 128
    }
  }
}
persistable: true
, name: "fc_0.b_0"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: FP32
      dims: 128
    }
  }
}
persistable: true
]


从输出结果中，可以看到通过调用 `fluid.layers.fc(input=x, size=128, bias_attr=True)` 后，会自动生成该层对应的可学习参数。其中 `fc_0.w_0` 是全连接层权重参数的名字，shape 为 (13, 128)，`fc_0.b_0` 是全连接层偏置参数的名字，shape 为 (128)。

### 输入输出 Tensor

整个神经网络的输入数据也是一个特殊的 Tensor，在这个 Tensor 中，一些维度的大小在定义模型时无法确定（通常包括：batch size，如果 mini-batch 之间数据可变，也会包括图片的宽度和高度等），在定义模型时需要占位。

Fluid 中使用 `fluid.layers.data` 来接收输入数据， `fluid.layers.data` 需要提供输入 Tensor 的形状信息，当遇到无法确定的维度时，相应维度指定为 None ，如下面的代码片段所示：

In [4]:
# 定义 input_x 的维度为 [3,None]，其中我们只能确定 input_x 的第一的维度为3，第二个维度未知，要在程序执行过程中才能确定
input_x = fluid.layers.data(name="x", shape=[3, None], dtype="int64")
print('shape of input_x: {}'.format(input_x.shape))

# batch size无需显示指定，框架会自动补充第0维为batch size，并在运行时填充正确数值
input_a = fluid.layers.data(name="a", shape=[3, 4], dtype='int64')
print('shape of input_a: {}'.format(input_a.shape))

# 若图片的宽度和高度在运行时可变，将宽度和高度定义为 None。
# shape的三个维度含义分别是：channel、图片的高度、图片的宽度
input_b = fluid.layers.data(name="image", shape=[3, None, None], dtype="float32")
print('shape of input_b: {}'.format(input_b.shape))

shape of input_x: (3L, -1L)
shape of input_a: (-1L, 3L, 4L)
shape of input_b: (3L, -1L, -1L)


### 常量 Tensor

Fluid 通过 `fluid.layers.fill_constant` 来实现常量 Tensor，用户可以指定 Tensor 的形状，数据类型和常量值。例如：

In [5]:
constant_data = fluid.layers.fill_constant(shape=[1], value=0, dtype='int64')
print(constant_data)

ones_data = fluid.layers.ones(shape=[1], dtype='int64')
print(ones_data)

zeros_data = fluid.layers.zeros(shape=[1], dtype='int64')
print(zeros_data)

name: "fill_constant_0.tmp_0"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: INT64
      dims: 1
    }
  }
}
persistable: false

name: "fill_constant_1.tmp_0"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: INT64
      dims: 1
    }
  }
}
persistable: false

name: "fill_constant_2.tmp_0"
type {
  type: LOD_TENSOR
  lod_tensor {
    tensor {
      data_type: INT64
      dims: 1
    }
  }
}
persistable: false



需要注意的是，上述定义的 tensor 并不具有值，它们仅表示将要执行的操作，具体输出数值将在 Executor 运行时得到。

In [6]:
c_tensor = fluid.layers.fill_constant(shape=[2, 4], value=2, dtype='float32')
exe = fluid.Executor(fluid.CPUPlace())
constant_value = exe.run(fetch_list=[c_tensor])
print(constant_value)

[array([[2., 2., 2., 2.],
       [2., 2., 2., 2.]], dtype=float32)]


## LoD-Tensosr

大多数的深度学习框架使用 Tensor 表示一个 mini-batch。

例如一个 mini-batch 中有`10`张图片，每幅图片大小为`32x32`，则这个 mini-batch 是一个`10x32x32`的 Tensor。

或者在处理NLP任务中，一个 mini-batch 包含 `N `个句子，每个字都用一个 `D` 维的 one-hot 向量表示，假设所有句子都用相同的长度 `L`，那这个 mini-batch 可以被表示为 `NxLxD` 的 Tensor。

上述两个例子中序列元素都具有相同大小，但是在许多情况下，训练数据是变长序列。基于这一场景，大部分框架采取的方法是确定一个固定长度，对小于这一长度的序列数据以 0 填充。

在Fluid中，由于**LoD-Tensor**的存在，我们不要求每个 mini-batch 中的序列数据必须保持长度一致，因此您不需要执行填充操作，也可以满足处理NLP等具有序列要求的任务需求。

Fluid 引入了一个索引数据结构（LoD）来将张量分割成序列。

### LoD 索引

#### 1. 句子组成的 mini-batch

假设一个 mini-batch 中有3个句子，每个句子中分别包含3个、1个和2个单词。我们可以用 `(3+1+2)xD` 维Tensor 加上一些索引信息来表示这个mini-batch:


```
3       1   2
| | |   |   | |
```

上述表示中，每一个` | `代表一个**D**维的词向量，数字`3，1，2`构成了 1-level LoD，即 `lod-level=1` 的Tensor。

#### 2. 递归序列

假设存在一个 mini-batch 中包含3个句子、1个句子和2个句子的文章，每个句子都由不同数量的单词组成，则这个 mini-batch 的样式可以看作：

```
3            1 2		# 句子数 
3   2  4     1 2  3		# 词数
||| || ||||  | || |||
```

表示的LoD信息为：

```
[[3，1，2]/*level=0*/，[3，2，4，1，2，3]/*level=1*/]
```

#### 3. 视频的mini-batch

在视觉任务中，时常需要处理视频和图像这些元素是高维的对象，假设现存的一个nimi-batch包含3个视频，分别有3个，1个和2个帧，每个帧都具有相同大小：640x480，则这个mini-batch可以被表示为：

```
3     1  2
口口口 口 口口
```

最底层 tensor 大小为`（3+1+2）x640x480`，每一个 `口` 表示一个640x480的图像

#### 4. 图像的mini-batch

在传统的情况下，比如有N个固定大小的图像的mini-batch，LoD-Tensor表示为:

```
1 1 1 1     1
口口口口 ... 口
```

在这种情况下，我们不会因为索引值都为1而忽略信息，仅仅把LoD-Tensor看作是一个普通的张量:

```
口口口口 ... 口
```

在Fluid中它们被表示为一个0-level LoD-Tensor。

### LoDTensor的偏移表示

为了快速访问基本序列，Fluid提供了一种偏移表示的方法——保存序列的开始和结束元素，而不是保存长度。

在上述例子中，您可以计算基本元素的长度：

```
3 2 4 1 2 3
```

将其转换为偏移表示：

```
0  3  5   9   10  12   15
   =  =   =   =   =    =
   3  2+3 4+5 1+9 2+10 3+12
```

所以我们知道第一个句子是从单词 0 到单词 3，第二个句子是从单词 3 到单词 5。

类似的，LoD 的顶层长度

```
3 1 2
```

可以被转化成偏移形式：

```
0 3 4   6
  = =   =
  3 3+1 4+2
```

因此该 LoD-Tensor 的偏移表示为：

```
0       3    4      6
  3 5 9   10   12 15
```

### LoD-Tensor

一个 LoD-Tensor 可以被看作是一个树的结构，树叶是基本的序列元素，树枝作为基本元素的标识。

在 Fluid 中 LoD-Tensor 的序列信息有两种表述形式：原始长度和偏移量。在 Paddle 内部采用偏移量的形式表述 LoD-Tensor，以获得更快的序列访问速度；在 python API中采用原始长度的形式表述 LoD-Tensor 方便用户理解和计算，并将原始长度称为： `recursive_sequence_lengths` 。

以上文提到的一个2-level LoD-Tensor为例：

```
3           1  2
3   2  4    1  2  3
||| || |||| |  || |||
```

* 以偏移量表示此 LoD-Tensor: `[ [0,3,4,6] , [0,3,5,9,10,12,15] ]`，
* 以原始长度表达此 Lod-Tensor：`recursive_sequence_lengths=[ [3 , 1 , 2] , [3, 2 , 4 , 1 , 2 , 3] ]`。

以文字序列为例： `[3,1,2]` 可以表示这个 mini-batch 中有3篇文章，每篇文章分别有3、1、2个句子，`[3,2,4,1,2,3]` 表示每个句子中分别含有3、2、4、1、2、3个字。

`recursive_seq_lens` 是一个双层嵌套列表，也就是列表的列表，最外层列表的size表示嵌套的层数，也就是 lod-level 的大小；内部的每个列表，对应表示每个` lod-level `下，每个元素的大小。

In [24]:
import numpy as np

data_np = np.array(
    [[1],[1],[1],
    [1],[1],
    [1],[1],[1],[1],
    [2],
    [3],[3],
    [3],[3],[3]]).astype('int64')
lod_t = fluid.create_lod_tensor(data_np, [[3,1,2] , [3,2,4,1,2,3]], fluid.CPUPlace())

print('LoD info: {}'.format(lod_t.lod()))
print('Shape: {}'.format(lod_t.shape()))

print('recursive sequence length: {}'.format(lod_t.recursive_sequence_lengths()))

# 查看lod-tensor嵌套层数
print('lod-level: {}'.format(len(lod_t.recursive_sequence_lengths())))

# 查看最基础元素个数
print('element count: {}'.format(sum(lod_t.recursive_sequence_lengths()[-1])))

Lod info: [[0L, 3L, 4L, 6L], [0L, 3L, 5L, 9L, 10L, 12L, 15L]]
Shape: [15L, 1L]
recursive sequence length: [[3L, 1L, 2L], [3L, 2L, 4L, 1L, 2L, 3L]]
lod-level: 2
element count: 15


关于 LoD-Tensor 的几个函数：

* `np.array(lod_tensor)`:  将 LoDTensor 转换为 numpy array
* `lod_tensor.lod()`: 获得LoD信息
* `lod_tensor.shape()`: 获取 Lod-Tensor 的形状
* `lod_tensot.recursive_sequence_lengths()`: output the length-based lod info
* `lod_tensor.has_valid_recursive_sequence_lengths()`: Check that the lod info is valid and match the outermost dimension of the LoDTensor data
* `lod_tensor.set_lod(arg0: List[List[int])→ None`:  the input lod is offset-based level-of-detail info
* `lod_tensor.set_recursive_sequence_lengths(arg0: List[List[int]]) → None`: the input recursive_sequence_lengths is length-based level-of-detail info

----

最后两个函数的讨论可以参考 [https://github.com/PaddlePaddle/Paddle/issues/10855](https://github.com/PaddlePaddle/Paddle/issues/10855)

LoD-Tensor 类的定义可以参考: [LoD-Tensor Class](https://github.com/PaddlePaddle/Paddle/blob/c9b4adf0d52f03ac1c6ffa309a519a3488f13f52/paddle/fluid/pybind/pybind.cc#L274)

LoD-Tensor 类中文文档：[lodtensor](http://paddlepaddle.org/documentation/docs/zh/1.2/api_cn/fluid_cn.html#lodtensor)

LoD-Tensor 使用说明：[LoD-Tensor使用说明](http://paddlepaddle.org/documentation/docs/zh/1.2/user_guides/howto/basic_concept/lod_tensor.html)

### LoD-Tensor 代码示例

本节代码将根据指定的级别 y-lod，扩充输入变量 x。本例综合了 LoD-Tensor 的多个重要概念，跟随代码实现，您将

* 直观理解Fluid中 `fluid.layers.sequence_expand` 的实现过程
* 掌握如何在 Fluid 中创建 LoD-Tensor
* 学习如何打印 LoDTensor 内容

#### 1. 定义计算过程

`layers.sequence_expand` 通过获取 y 的 lod 值对 x 的数据进行扩充，关于` fluid.layers.sequence_expand` 的函数说明，请先阅读 [fluid.layers.sequence_expand](http://paddlepaddle.org/documentation/docs/zh/1.2/api_cn/layers_cn.html#sequence-expand) 。

序列扩充代码实现：

```
x = fluid.layers.data(name='x', shape=[1], dtype='float32', lod_level=0)
y = fluid.layers.data(name='y', shape=[1], dtype='float32', lod_level=1)
out = fluid.layers.sequence_expand(x=x, y=y, ref_level=0)
```

In [27]:
# 由于该 notebook 中含有重复的变量定义，为了不报错，这里将变量和操作定义在单独的一个 Program 中
example_program = fluid.Program()

with fluid.program_guard(example_program, fluid.Program()):
    x = fluid.layers.data(name='x', shape=[1], dtype='float32', lod_level=0)
    y = fluid.layers.data(name='y', shape=[1], dtype='float32', lod_level=1)
    out = fluid.layers.sequence_expand(x=x, y=y, ref_level=0)

说明：输出LoD-Tensor的维度仅与传入的真实数据维度有关，在定义网络结构阶段为x、y设置的shape值，仅作为占位，并不影响结果。

#### 2. 创建Executor

```
place = fluid.CPUPlace()
exe = fluid.Executor(place)
exe.run(fluid.default_startup_program())
```

In [28]:
place = fluid.CPUPlace()
exe = fluid.Executor(place)
exe.run(fluid.default_startup_program())

[]

#### 3. 准备数据

这里我们调用 `fluid.create_lod_tensor` 创建 `sequence_expand` 的输入数据，通过定义 `y_d` 的 LoD 值，对 `x_d` 进行扩充。其中，输出值只与 `y_d` 的 LoD 值有关，`y_d` 的 data 值在这里并不参与计算，维度上与 LoD[-1] 一致即可。

`fluid.create_lod_tensor()` 的使用说明请参考 [api_fluid_create_lod_tensor](http://paddlepaddle.org/documentation/docs/zh/1.2/api_cn/fluid_cn.html#paddle.fluid.create_lod_tensor) 。

实现代码如下：

```
x_d = fluid.create_lod_tensor(np.array([[1.1],[2.2],[3.3],[4.4]]).astype('float32'), [[1,3]], place)
y_d = fluid.create_lod_tensor(np.array([[1.1],[1.1],[1.1],[1.1],[1.1],[1.1]]).astype('float32'), [[1,3], [2,1,2,1]],place)
```

In [29]:
x_d = fluid.create_lod_tensor(np.array([[1.1],[2.2],[3.3],[4.4]]).astype('float32'), [[1,3]], place)
y_d = fluid.create_lod_tensor(np.array([[1.1],[1.1],[1.1],[1.1],[1.1],[1.1]]).astype('float32'), [[1,3], [2,1,2,1]],place)

#### 4.执行运算

在Fluid中，LoD>1 的 Tensor 与其他类型的数据一样，使用 feed 定义数据传入顺序。此外，由于输出 results 是带有 LoD 信息的 Tensor，需在 `exe.run( )` 中添加 `return_numpy=False` 参数，获得 LoD-Tensor 的输出结果。

```
results = exe.run(fluid.default_main_program(),
                  feed={'x':x_d, 'y': y_d },
                  fetch_list=[out],return_numpy=False)
```

In [30]:
results = exe.run(example_program,
                  feed={'x':x_d, 'y': y_d },
                  fetch_list=[out],return_numpy=False)

#### 5. 查看LodTensor结果

由于 LoDTensor 的特殊属性，无法直接 print 查看内容，常用操作时将 LoD-Tenso r作为网络的输出 fetch 出来，然后执行 `numpy.array(lod_tensor)`, 就能转成 numpy array：

```
np.array(results[0])
```

In [33]:
np.array(results[0])

array([[1.1],
       [2.2],
       [3.3],
       [4.4],
       [2.2],
       [3.3],
       [4.4],
       [2.2],
       [3.3],
       [4.4]], dtype=float32)

#### 6. 查看序列长度

可以通过查看序列长度得到 LoDTensor 的递归序列长度：

```
results[0].recursive_sequence_lengths()
```

In [34]:
results[0].recursive_sequence_lengths()

[[1L, 3L, 3L, 3L]]

#### 7. 示例完整代码

您可以运行下列完整代码，观察输出结果：

> 注意：如果是在该 notebook 中运行下面的代码，需要重启 kernel，然后再单独运行下面的代码，否则会出现问题。因为 default_main_program 中包含有上面运行生成的变量和操作！！！

In [1]:
# 加载库
import paddle
import paddle.fluid as fluid
import numpy as np
# 定义前向计算
x = fluid.layers.data(name='x', shape=[1], dtype='float32', lod_level=0)
y = fluid.layers.data(name='y', shape=[1], dtype='float32', lod_level=1)
out = fluid.layers.sequence_expand(x=x, y=y, ref_level=0)
# 定义运算场所
place = fluid.CPUPlace()
# 创建执行器
exe = fluid.Executor(place)
exe.run(fluid.default_startup_program())
# 创建LoDTensor
x_d = fluid.create_lod_tensor(np.array([[1.1], [2.2],[3.3],[4.4]]).astype('float32'), [[1,3]], place)
y_d = fluid.create_lod_tensor(np.array([[1.1],[1.1],[1.1],[1.1],[1.1],[1.1]]).astype('float32'), [[1,3], [1,2,1,2]], place)
# 开始计算
results = exe.run(fluid.default_main_program(),
                  feed={'x':x_d, 'y': y_d },
                  fetch_list=[out],return_numpy=False)
# 输出执行结果
print("The data of the result: {}.".format(np.array(results[0])))
# 输出 result 的序列长度
print("The recursive sequence lengths of the result: {}.".format(results[0].recursive_sequence_lengths()))
# 输出 result 的 LoD
print("The LoD of the result: {}.".format(results[0].lod()))

The data of the result: [[1.1]
 [2.2]
 [3.3]
 [4.4]
 [2.2]
 [3.3]
 [4.4]
 [2.2]
 [3.3]
 [4.4]].
The recursive sequence lengths of the result: [[1L, 3L, 3L, 3L]].
The LoD of the result: [[0L, 1L, 4L, 7L, 10L]].


`fluid.layers.sequence_expand()` 如何理解，API 参考 [sequence_expand](http://paddlepaddle.org/documentation/docs/zh/1.2/api_cn/layers_cn.html#sequence-expand)

看一下 API 中的例子

```
 例1
        x is a LoDTensor:
            x.lod  = [[2,        2]]
            x.data = [[a], [b], [c], [d]]
            x.dims = [4, 1]

        y is a LoDTensor:
            y.lod = [[2,    2],
                     [3, 3, 1, 1]]

        ref_level: 0

        then output is a 1-level LoDTensor:
            out.lod =  [[2,        2,        2,        2]]
            out.data = [[a], [b], [a], [b], [c], [d], [c], [d]]
            out.dims = [8, 1]
```

首先假如这是一个 batch 的数据信息。根据 x.lod 和 `ref_level` 将 x 分为两部分，第一部分为 `[a], [b]`，第二部分为 `[c], [d]`，然后在 x 的基础之上扩展一个 lod-level，也就是在 x 中添加一个 `lod-level=0` 给 x 的 lod。此时， y 的 lod-level=0 的列表为 [2, 2]，所以最终结果由 2 个 x 的第一部分， 2 个 x 的第二部分组成，即

```
[[2,        2,        2,        2]]
[[a], [b], [a], [b], [c], [d], [c], [d]]
```

```
* 例2
        x is a Tensor:
            x.data = [[a], [b], [c]]
            x.dims = [3, 1]

        y is a LoDTensor:
            y.lod = [[2, 0, 3]]

        ref_level: -1

        then output is a Tensor:
            out.data = [[a], [a], [c], [c], [c]]
            out.dims = [5, 1]
```

例 2 原理是相同的，只是 x 的 `lod-level=0`，y 的 `lod-level=1`。

同理，可以得到上面代码的结果应该为：

```
[1      3                    3                    3                  ]
[[1.1], [2.2], [3.3], [4.4], [2.2], [3.3], [4.4], [2.2], [3.3], [4.4]]
```

## Reference

[1] [LoD-Tensor使用说明](http://paddlepaddle.org/documentation/docs/zh/1.2/user_guides/howto/basic_concept/lod_tensor.html)

[2] [Fluid编程指南](http://paddlepaddle.org/documentation/docs/zh/1.2/beginners_guide/programming_guide/programming_guide.html)

[3] [LoDTensor类源码](https://github.com/PaddlePaddle/Paddle/blob/c9b4adf0d52f03ac1c6ffa309a519a3488f13f52/paddle/fluid/pybind/pybind.cc#L274)

[4] [create_lod_tensor API](http://paddlepaddle.org/documentation/docs/zh/1.2/api_cn/fluid_cn.html#create-lod-tensor)

[5] [LoDTensor class API](http://paddlepaddle.org/documentation/docs/zh/1.2/api_cn/fluid_cn.html#lodtensor)

[6] [sequence_expand API](http://paddlepaddle.org/documentation/docs/zh/1.2/api_cn/layers_cn.html#sequence-expand)